In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, Input, Flatten, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

In [ ]:
diabetesdataurl='app\\static\\Datasets\\diabetes.csv'
diabetesdata_cp= pd.read_csv(diabetesdataurl)
diabetesdata=diabetesdata_cp.copy(deep=True)
diabetesdata

In [ ]:
diabetesdata[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = diabetesdata[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)
diabetesdata.dropna(inplace=True,axis=0)
# diabetesdata['Glucose'].fillna(diabetesdata['Glucose'].mean(),inplace=True)
# diabetesdata['BloodPressure'].fillna(diabetesdata['BloodPressure'].mean(),inplace=True)
# diabetesdata['SkinThickness'].fillna(diabetesdata['SkinThickness'].median(),inplace=True)
# diabetesdata['Insulin'].fillna(diabetesdata['Insulin'].median(),inplace=True)
# diabetesdata['BMI'].fillna(diabetesdata['BMI'].median(),inplace=True)

In [ ]:
X=diabetesdata.drop(['Outcome','BloodPressure', 'SkinThickness'],axis=1)
# X=diabetesdata.drop(['Outcome'],axis=1)

Y=diabetesdata['Outcome']
X.shape

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = Sequential()
model.add(Input(6,))

model.add(Dense(units=30, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=90, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=180, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])



In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
path_to_save_model='./models'
early_stop=EarlyStopping(monitor="val_loss",patience=100)#this means if after 10 epochs the model is not improving(ie:validation loss is not reducing) then the training stops 
checkpointsaver=ModelCheckpoint(path_to_save_model,monitor="val_accuracy",mode='max',save_best_only=True,save_freq='epoch',verbose=1)#reason for these parameters is to enable saving the model on highest validation accuracy on an epoch


In [ ]:
model.fit(X_train, Y_train, epochs=200, validation_split=0.2, callbacks=[checkpointsaver,early_stop])

In [ ]:
model.evaluate(X_test,Y_test,batch_size=100)

In [ ]:
model=load_model('./models')

In [ ]:
model.predict(X_test)

In [ ]:
np.shape(model.predict(X_test))

In [ ]:
Y_test

In [ ]:
pregnanciesno=1
glucose=137
bmi=45
insulin=200
Age=33
bloodpressure=300
# SkinThickness=40
pedigree=2.288
features=[pregnanciesno,glucose,bloodpressure,insulin,bmi,pedigree,Age]
features=np.array(features)
features=features.reshape(1,-1)
# features=scaler.transform(features)
predictionvalue=model.predict(features)
prediction=model.predict_proba(features)
print(predictionvalue, prediction)
if predictionvalue==1:
    print ('You have higher chances of diabetes')
else:
    print ('Minimal chances of diabetes')